In [1]:
import numpy as np

In [4]:
org_npy = np.load('data/fern.npy', allow_pickle=True)

In [5]:
for i in org_npy:
    print(i.shape)

(63, 256)
(256,)
(256, 256)
(256,)
(256, 256)
(256,)
(256, 256)
(256,)
(256, 256)
(256,)
(319, 256)
(256,)
(256, 256)
(256,)
(256, 256)
(256,)
(256, 256)
(256,)
(283, 128)
(128,)
(128, 3)
(3,)
(256, 1)
(1,)


In [6]:
def encode_nerf(nerf_npy):
    new_np = np.zeros((10, 256, 256))
    input_layer = np.array([], dtype=np.float32)
    bias_layer = np.array([], dtype=np.float32)

    cn_i = 0
    
    for i in range(0, len(nerf_npy), 2):
        weight_shape = nerf_npy[i].shape
        if weight_shape == (256, 256):
            new_np[cn_i, :, :] = nerf_npy[i]
            bias_layer = np.append(bias_layer, nerf_npy[i+1].flatten())
            cn_i += 1
        elif weight_shape == (319, 256):
            new_np[cn_i, :, :] = nerf_npy[i][63:, :]
            input_layer = np.append(input_layer, nerf_npy[i][:63, :].flatten())
            bias_layer = np.append(bias_layer, nerf_npy[i+1].flatten())
            cn_i += 1
        elif weight_shape == (283, 128):
            input_layer = np.append(input_layer, nerf_npy[i][:27, :].flatten())
            bias_layer = np.append(bias_layer, nerf_npy[i][27:, :].flatten())
            bias_layer = np.append(bias_layer, nerf_npy[i+1].flatten())
        else:
            input_layer = np.append(input_layer, nerf_npy[i].flatten())
            bias_layer = np.append(bias_layer, nerf_npy[i+1].flatten())
    # print(input_layer.shape[0]/256)
    # print(bias_layer.shape[0]/256)
    # print((input_layer.shape[0] + bias_layer.shape[0])/256)

    padding = np.zeros((256*256) - input_layer.size)
    input_layer = np.concatenate((input_layer, padding))
    
    padding = np.zeros((256*256) - bias_layer.size)
    bias_layer = np.concatenate((bias_layer, padding))
    
    new_np[8, :, :] = input_layer.reshape(256, 256)
    new_np[9, :, :] = bias_layer.reshape(256, 256)
    
    return new_np


def decode_nerf(new_np):
    original_params = []
    
    # input_layer와 bias_layer 복원
    input_layer_flat = new_np[8].flatten()
    bias_layer_flat = new_np[9].flatten()
    
    input_i = 0
    bias_i = 0
    
    # Layer 0 복원
    first_weight = input_layer_flat[input_i:input_i + 63*256].reshape(63, 256)
    original_params.append(first_weight)
    input_i += 63 * 256
    
    first_bias = bias_layer_flat[bias_i:bias_i + 256]
    original_params.append(first_bias)
    bias_i += 256
    
    # Layer 1~8 복원
    for cn_i in range(0, 8):
        weight = new_np[cn_i]
        if cn_i == 4:
            # (319, 256) 레이어 복원
            w5_2 = input_layer_flat[input_i:input_i + 63*256].reshape(63, 256)
            input_i += 63 * 256
            original_params.append(np.vstack((w5_2, weight)))
        else:
            original_params.append(weight)

        bias = bias_layer_flat[bias_i:bias_i + 256]
        bias_i += 256
        original_params.append(bias)
    
    # Layer 9 복원
    w9_1 = input_layer_flat[input_i:input_i + 27*128].reshape(27, 128)
    input_i += 27*128
    w9_2 = bias_layer_flat[bias_i:bias_i + 256*128].reshape(256, 128)
    bias_i += 256*128
    original_params.append(np.vstack((w9_1, w9_2)))
    
    b9 = bias_layer_flat[bias_i:bias_i + 128]
    bias_i += 128
    original_params.append(b9)
    
    # color Layer 복원
    wc = input_layer_flat[input_i:input_i + 128*3].reshape(128, 3)
    input_i += 128*3
    original_params.append(wc) 
    
    bc = bias_layer_flat[bias_i:bias_i + 3]
    bias_i += 3
    original_params.append(bc)

    # density Layer 복원
    wd = input_layer_flat[input_i:input_i+256].reshape(256, 1)
    original_params.append(wd)
    bd = bias_layer_flat[bias_i:bias_i+1]
    original_params.append(bd)
    
    return np.array(original_params, dtype=object)

def compare_params(original, decoded):
    for i, (orig_param, dec_param) in enumerate(zip(original, decoded)):
        if not np.array_equal(orig_param, dec_param):
            print(f"Mismatch found at parameter {i}")
            print(f"Original shape: {orig_param.shape}, Decoded shape: {dec_param.shape}")
            print(f"Original parameter: \n{orig_param}")
            print(f"Decoded parameter: \n{dec_param}")
            return False
    print("All parameters match.")
    return True

In [7]:
enc_npy = encode_nerf(org_npy)
dec_npy = decode_nerf(enc_npy)
compare_params(org_npy, dec_npy)

All parameters match.


True

In [93]:
np.save('enc_fern.npy', enc_npy)

In [94]:
np.save('dec_fern.npy', dec_npy)

In [36]:
def encode10_3(org_npy):
    t_shape = (512, 392)
    t_size = 512 * 392
    extra_size = 2048

    layers_6_9 = [org_npy[6], org_npy[7], org_npy[8][:142, :], org_npy[9][:138, :]]
    flattened_6_9 = np.concatenate([layer.flatten() for layer in layers_6_9])

    channel_3 = flattened_6_9[:t_size-extra_size]
    channel_3 = np.pad(channel_3, (0, extra_size)).reshape(t_shape)
    
    remain_6_9 =  flattened_6_9[t_size-extra_size:]
    remain1 = remain_6_9[:extra_size]
    remain2 = remain_6_9[extra_size:]
    
    # 첫 번째 채널에 0, 1, 2 레이어
    channel_1 = np.concatenate([layer.flatten() for layer in org_npy[0:3]])
    channel_1 = np.concatenate([channel_1, remain1])
    channel_1 = np.pad(channel_1, (0, 2048)).reshape(t_shape)

    # 두 번째 채널에 3, 4, 5 레이어
    channel_2 = np.concatenate([layer.flatten() for layer in org_npy[3:6]])
    channel_2 = np.concatenate([channel_2, remain2])               
    channel_2 = np.pad(channel_2, (0, 2048)).reshape(t_shape)
    
    # 결과 배열 생성
    encoded_npy = np.stack((channel_1, channel_2, channel_3), axis=0)

    return encoded_npy

def decode10_3(encoded_npy):
    t_shape = (256, 256)
    t_size = 256 * 256
    def unflatten_and_extract(layers, target_shape):
        flattened = layers.flatten()
        extracted_layers = []
        remain_idx = 0
        for i in range(0, len(flattened), t_size):
            if i+t_size < len(flattened):
                layer = flattened[i:i+t_size].reshape(target_shape)
                extracted_layers.append(layer)
            else:
                remain_idx = i
        return extracted_layers, flattened[remain_idx:]
    
    # 첫 번째 채널에서 0, 1, 2 레이어 복원
    layers_0_2, remain1 = unflatten_and_extract(encoded_npy[0], t_shape)
    remain1 = remain1[:2048]

    # 두 번째 채널에서 3, 4, 5 레이어 복원
    layers_3_5, remain2 = unflatten_and_extract(encoded_npy[1], t_shape)
    remain2 = remain2[:2048]

    # 세 번째 채널에서 6, 7, 8, 9 레이어 복원
    flattened_6_9 = encoded_npy[2].flatten()

    layer_6 = flattened_6_9[:t_size].reshape(t_shape)
    layer_7 = flattened_6_9[t_size:2*t_size].reshape(t_shape)
    
    layer_8 = flattened_6_9[2*t_size:2*t_size + 256*142]
    layer_8 = np.pad(layer_8, (0, 256*114), 'constant', constant_values=0).reshape(t_shape)
    
    layer_9 = flattened_6_9[2*t_size + 256*142:-2048]
    layer_9 = np.concatenate([layer_9, remain1, remain2])

    layer_9 = np.pad(layer_9, (0, 256*118), 'constant', constant_values=0).reshape(t_shape)

    layers_0_2.extend(layers_3_5)
    layers_0_2.extend([layer_6, layer_7, layer_8, layer_9])

    return np.array(layers_0_2)

en3 = encode10_3(enc_npy)
de3 = decode10_3(en3)
compare_params(enc_npy, de3)

All parameters match.


True